In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('desafio-base1/pacientes.csv', parse_dates=['DataNasc'], encoding='iso-8859-1', quotechar='"', delimiter='|')
df.shape

(783, 18)

In [3]:
df.head(3).T

,0,1,2
Código,0,1,2
Nome,Pedro Henrique Porto,João Vitor da Paz,Theo Nunes
DataNasc,NaT,1996-01-01 00:00:00,1959-01-01 00:00:00
Sexo,M,M,NaN
Estado,RJ,AP,NaN
Endereco,"Quadra Caroline Dias, 482, Salgado Filho","Setor Nunes, Santa Lúcia",NaN
Cidade,Ramos,Nogueira,NaN
CEP,31924-681,28517-970,NaN
Naturalidade,Campos do Amparo,Araújo,Lopes
Telefone,(021) 9313-9128,(021) 5202 2566,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Código            783 non-null    int64         
 1   Nome              783 non-null    object        
 2   DataNasc          635 non-null    datetime64[ns]
 3   Sexo              632 non-null    object        
 4   Estado            632 non-null    object        
 5   Endereco          632 non-null    object        
 6   Cidade            632 non-null    object        
 7   CEP               632 non-null    object        
 8   Naturalidade      643 non-null    object        
 9   Telefone          615 non-null    object        
 10  TipoTelefone      615 non-null    object        
 11  Profissao         629 non-null    object        
 12  Pai               622 non-null    object        
 13  Mae               633 non-null    object        
 14  Conjuge           639 non-

In [5]:
# Trantando recurso "Sexo"
# Testei uma ideia originalmente, mas não me pareceu foi correta
# df['Sexo'] = df['Sexo'].astype(str).str.lower()
# Pelo que achei melhor utilizar list comprehension, desta forma se tiver algum outro caracter não correto ele receberá o valor NaN
#print(df['Sexo'].isna().sum())
df['Sexo'] = ['m' if x == 'M' else 'f' if x == 'F' else np.nan for x in df['Sexo']]
#print(df['Sexo'].isna().sum())
# Verifiquei que a quantidade de NaN ficou imutável após a mudança

In [6]:
# Trantando recurso "Estado"
teste = df['Estado'].str.contains(r'\b[A-Z]{2}')
print(teste.value_counts())
# Verifiquei se alguma informação não se adequava ao padrão estado (duas letras maiusculas), como o valore de True coincide com a quantidade de valores não nulos da coluna, logo todos os registros estão corretamente formatados

True    632
Name: Estado, dtype: int64


In [7]:
# Trantando recurso "CEP"
# Primeiro verifiquei a quantidade de NaN para garantir que ele continuasse o mesmo
# Em seguida mapiei os registros que possuem o caracter '-'
# print(df['CEP'].isna().sum())
rows_with_dashes = df['CEP'].str.contains('-')
df['CEP'] = [df['CEP'][i] if x == True else df['CEP'][i][:5]+'-'+df['CEP'][i][5:] if x == False else np.nan for i, x in enumerate(rows_with_dashes)]
# print(df['CEP'].isna().sum())

In [8]:
# Tratar recurso "EstadoCivil"
print(df['EstadoCivil'].value_counts())
# Obtendo as seguintes categorias:
# CA = casado = ma
# ES = união estável = st
# VI = Viúvo = wi
# SE = Separado = se

CA    187
ES    170
VI    145
SE    132
Name: EstadoCivil, dtype: int64


In [9]:
df['EstadoCivil'].replace({'CA': 'ms', 'ES': 'st', 'VI': 'wi','SE': 'se'}, inplace=True)

In [10]:
# Tratar recurso "Cor"
print(df['Cor'].value_counts())
#B = branca = wh
#A = amarela = ye
#P = parda = br
#N = negra = bl
#I = indegena = br?

B    132
A    131
P    127
N    127
I    114
Name: Cor, dtype: int64


In [11]:
df['Cor'].replace({'B': 'wh', 'A': 'ye', 'P': 'br','N': 'bl', 'I': 'br'}, inplace=True)

In [12]:
#Tratar recurso 'Endereco'
#Criando novas colunas relativas as informações dp endereço do paciente 
df['address'] = np.nan
df['number'] = np.nan
df['complement'] = np.nan
df['neighborhood'] = np.nan
df['country'] = "BR"

In [13]:
# Separando as informações e distribuindo de acordo ao seu tipo
row_with_adress = df['Endereco'].str.split(',')
df['number'] = [np.nan if str(x) == 'nan' else x[1].strip() if len(x) == 3 else np.nan for x in row_with_adress]
df['neighborhood'] = [np.nan if str(x) == 'nan' else x[1].strip() if len(x) == 2 else x[2].strip() for x in row_with_adress]
df['address'] = [np.nan if str(x) == 'nan' else x[0] for x in row_with_adress]

In [14]:
#Fazendo analise dos valores na serie TipoTelefone
print(df['TipoTelefone'].value_counts())

C    225
T    195
R    195
Name: TipoTelefone, dtype: int64


In [15]:
#Criando as colunas relativas ao tipo de phone e tratando as series 'Telefone' e 'TipoTelefone'
df['mobile_phone'] = np.nan
df['home_phone'] = np.nan
df['office_phone'] = np.nan

In [16]:
#Extraindo apenas os valores numericos dos numeros de telefone
df['Telefone'] = df['Telefone'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int).astype('str')

In [17]:
#Verificando o tipo de telefone T, trabalho
rows_t = df['TipoTelefone'].str.contains('T', na=False)
df['office_phone'] = '(' + df['Telefone'][rows_t].str[-10:-8] + ')' + df['Telefone'][rows_t].str[-8:-4] + '-' + df['Telefone'][rows_t].str[-4:]
#Verificando o tipo de telefone C, celular
rows_t = df['TipoTelefone'].str.contains('C', na=False)
df['mobile_phone'] = '(' + df['Telefone'][rows_t].str[:2] + ')9' + df['Telefone'][rows_t].str[-8:-4] + '-' + df['Telefone'][rows_t].str[-4:]
#Verificando o tipo de telefone R, residencial
rows_t = df['TipoTelefone'].str.contains('R', na=False)
df['home_phone'] = '(' + df['Telefone'][rows_t].str[-10:-8] + ')' + df['Telefone'][rows_t].str[-8:-4] + '-' + df['Telefone'][rows_t].str[-4:]

In [18]:
# Colocando o nome das colunas do arquivo de acordo a documentacção da iClinic
df = df.rename(
    columns = {
        "Código":"patient_id",
        "Nome": "name",
        "DataNasc":"birth_date",
        "Sexo": "gender",
        "Estado": "state",
        "Cidade": "city",
        "CEP": "zip_code",
        "Naturalidade": "birth_place",
        "Profissao": "occupation",
        "Pai": "patientrelatedness_father_names",
        "Mae": "patientrelatedness_mother_names",
        "Cor": "ethnicity",
        "EstadoCivil": "marital_status",
    }
)

In [22]:
#Adiconando as colunas faltantes
df['gender'] = np.nan
df['cpf'] = np.nan
df['rg'] = np.nan
df['rg_issuer'] = np.nan #na documentação estava repetindo 'rg'
df['email'] = np.nan
df['email_secondary'] = np.nan
df['birth_state'] = np.nan
df['picture_filename'] = np.nan
df['religion'] = np.nan
df['education'] = np.nan
df['responsible'] = np.nan
df['cns'] = np.nan #na documentação estava não existe 'sms'
df['died'] = np.nan
df['death_info'] = np.nan
df['nationality'] = np.nan 
df['indication'] = np.nan
df['indication_observation'] = np.nan
df['active'] = np.nan 
df['receive_email'] = np.nan
df['observation'] = np.nan
df['healthinsurance_pack'] = np.nan 
df['tag_names'] = np.nan
df['tag_physician_id'] = np.nan

In [23]:
df =  df.loc[:, ["patient_id","name","birth_date","gender","cpf","rg","rg_issuer","mobile_phone","home_phone","office_phone","email","email_secondary","birth_place","birth_state","zip_code","address","number","complement","neighborhood","city","state","country","picture_filename","ethnicity","marital_status","religion","occupation","education","responsible","cns","died","death_info","nationality","indication","indication_observation","active","receive_email","observation","healthinsurance_pack","patientrelatedness_mother_names","patientrelatedness_father_names","tag_names","tag_physician_id"]]

In [24]:
df.to_csv('patient.csv',index=False, encoding='utf-8')
